# Natural Language Processing HW-1 Report

In [1]:
#Author: Wei-Cheng Chen
#Student ID: 0816065
#HW ID: hw1
#Due Date: 03/17/2022

## Import

In [2]:
import pandas as pd
import numpy as np
import numpy.matlib 
import random

## Data accessing

In [3]:
p = pd.read_csv('input.txt',delimiter=" ", header=None, index_col=0)
print(p)

                            1         2            3           4
0                                                               
acknowledgement     knowledge   learned  predicament     knowing
hygiene                hygene   genetic         high      hugely
mischievous       mischevious  mischief      devious      sneaky
vacuum                vaccuum    vaccum      upright     cleaner
maintenance      maintainance   service        parts  sustenance


In [4]:
c1 = pd.read_csv('costs1.csv',index_col=0)
c2 = pd.read_csv('costs2.csv',index_col=0)

## Print function

In [5]:
def printit(w1,w2,bt,cost) :
    w1_p = list(w1)
    w2_p = list(w2)
    for i in range(len(bt)) :
        if bt[i] == 'd' :
            w2_p.insert(i, '*')
        elif bt[i] == 'i' :
            w1_p.insert(i, '*')
    for char in w1_p :
        print(char,end=" ")
    print()
    print('| '*max(len(w1_p),len(w2_p)))
    for char in w2_p :
        print(char,end=" ")
    print()
    for char in bt:
        print(char,end=" ")
    print()
    print("cost: ", cost)
    print()

## Levenshtein 

In [6]:
def backtrace(m, colli) :
    trace = []
    def val(x,y) :
        if x >= 0 and y >= 0 :
            return m[x,y]
        else :
            return -10
        
    x , y = m.shape 
    x = x-1 ; y = y-1
    while x >= 0 and y >= 0 :
        check = [ 1 if val(x-1,y-1) == m[x,y] - 2 else 0 , 1 if (val(x-1,y-1) == m[x,y] and (x,y) in colli)  else 0,1 if val(x-1,y) == m[x,y] - 1 else 0 , 1 if val(x,y-1) == m[x,y] - 1 else 0 ]
        if sum(check) != 0 :
            # randomly selected
            select = random.choice([ i for i,j in enumerate(check) if j==1])
        else :
            break
        if select == 0 :
            trace.append('s')
            x = x-1 ; y = y-1
        elif select == 1 :
            trace.append('n')
            x = x-1 ; y = y-1
        elif select == 2 :
            trace.append('d')
            x = x-1
        elif select == 3 :
            trace.append('i')
            y = y-1
    
    trace.reverse()
    return trace

def mineditdis(w1,w2) :
    colli = []
    m = np.matlib.zeros((len(w1)+1,len(w2)+1))
    m[0] = [ i for i in range(len(w2)+1) ] 
    m.T[0] = [ i for i in range(len(w1)+1) ] 
    for i in range(len(w1)) :
        for j in range(len(w2)) :
            m[i+1,j+1] = min( m[i,j+1]+1 , m[i+1,j]+1 , m[i,j]+ c1[ w2[j] ][  w1[i] ] )
            if w2[j] ==  w1[i] :
                colli.append((i+1,j+1))
    
    bt = backtrace(m, colli)    
    
    printit(w1,w2,bt,m[len(w1),len(w2)])
    
    return m
    
for tar in p.index :
    for word in p.loc[tar] :
        mineditdis(word,tar)

* * k n o w l e d g e * * * * 
| | | | | | | | | | | | | | | 
a c k n o w l e d g e m e n t 
i i n n n n n n n n n i i i i 
cost:  6.0

l e a r * * n * * * e d * * * * * * 
| | | | | | | | | | | | | | | | | | 
* * a * c k n o w l e d g e m e n t 
d d n d i i n i i i n n i i i i i i 
cost:  14.0

* * * * * p r * e d i c a * m e n t 
| | | | | | | | | | | | | | | | | | 
a c k n o w * l e d * g * e m e n t 
i i i i i s d i n n d s d i n n n n 
cost:  14.0

* * k n o w * * * * * * * i n * g 
| | | | | | | | | | | | | | | | | 
a c k n o w l e d g e m e * n t * 
i i n n n n i i i i i i i d n i d 
cost:  12.0

h y g * e n e 
| | | | | | | 
h y g i e n e 
n n n i n n n 
cost:  1.0

* * g * e n e t i c 
| | | | | | | | | | 
h y g i e n e * * * 
i i n i n n n d d d 
cost:  6.0

h * * i * * g h 
| | | | | | | | 
h y g i e n e * 
n i i n i i s d 
cost:  7.0

h u * g * * * e l y 
| | | | | | | | | | 
h * y g i e n e * * 
n d i n i i i n d d 
cost:  7.0

m i s c h * e v i o u s 
| | | | | | | | | | 

## Weighted 

In [7]:
def backtrace_w(m, colli, weight) :
    trace = []
    def val(x,y) :
        if x >= 0 and y >= 0 :
            return m[x,y]
        else :
            return -10
        
    x , y = m.shape 
    x = x-1 ; y = y-1
    while x >= 0 and y >= 0 :
        check = [ 1 if ( (x,y) not in colli and val(x-1,y-1) == m[x,y] - weight.get((x,y),-100) ) else 0 , 1 if ( val(x-1,y-1) == m[x,y] and (x,y) in colli )  else 0,1 if val(x-1,y) == m[x,y] - 1 else 0 , 1 if val(x,y-1) == m[x,y] - 1 else 0 ]
        if sum(check) != 0 :
            # randomly selected
            select = random.choice([ i for i,j in enumerate(check) if j==1])
        else :
            break
        if select == 0 :
            trace.append('s')
            x = x-1 ; y = y-1
        elif select == 1 :
            trace.append('n')
            x = x-1 ; y = y-1
        elif select == 2 :
            trace.append('d')
            x = x-1
        elif select == 3 :
            trace.append('i')
            y = y-1
    
    trace.reverse()
    return trace

def mineditdis_w(w1,w2) :
    
    colli = []
    weight = {}
    m = np.matlib.zeros((len(w1)+1,len(w2)+1))
    m[0] = [ i for i in range(len(w2)+1) ] 
    m.T[0] = [ i for i in range(len(w1)+1) ] 
    for i in range(len(w1)) :
        for j in range(len(w2)) :
            m[i+1,j+1] = min( m[i,j+1]+1 , m[i+1,j]+1 , m[i,j]+ c2[ w2[j] ][  w1[i] ] )
            if w2[j] ==  w1[i] :
                colli.append((i+1,j+1))
            else :
                weight[(i+1,j+1)] = c2[ w2[j] ][  w1[i] ]
    
    bt = backtrace_w(m, colli, weight)    
    
    printit(w1,w2,bt,m[len(w1),len(w2)])
    
    return m

for tar in p.index :
    for word in p.loc[tar] :
        mineditdis_w(word,tar)

* * k n o w l e d g e * * * * 
| | | | | | | | | | | | | | | 
a c k n o w l e d g e m e n t 
i i n n n n n n n n n i i i i 
cost:  6.0

l e a r * * n * * e d * * * * * * 
| | | | | | | | | | | | | | | | | 
* * a c k n o w l e d g e m e n t 
d d n s i i s i i n n i i i i i i 
cost:  12.0

p r e * d i * c * a m e * n t 
| | | | | | | | | | | | | | | 
a c k n o w l e d g e m e n t 
s s s i s s i s i s s s i n n 
cost:  6.0

* * * k n o w * i n * g * * * 
| | | | | | | | | | | | | | | 
a c k n o w l e d g e m e n t 
i i i s s s s i s s i s i i i 
cost:  9.0

h y * g e n e 
| | | | | | | 
h y g i e n e 
n n i s n n n 
cost:  1.0

g e n * * e t i c 
| | | | | | | | | 
* h y g i e * n e 
d s s i i n d s s 
cost:  4.0

* * h i * g h 
| | | | | | | 
h y g i e n e 
i i s n i s s 
cost:  3.0

h * u g e l y * 
| | | | | | | | 
h y g i * e n e 
n i s s d s s i 
cost:  3.0

m i s c h e v * i o u s 
| | | | | | | | | | | | 
m i s c h * i e v o u s 
n n n n n d s i s n n n 
cost:  2.0

m i s c h i e f

# My result

## Levenshtein 

In [8]:
print(mineditdis("acknowledgement","knowledge"))

a c k n o w l e d g e m e n t 
| | | | | | | | | | | | | | | 
* * k n o w l e d g e * * * * 
d d n n n n n n n n n d d d d 
cost:  6.0

[[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
 [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
 [ 2.  3.  4.  5.  6.  7.  8.  9. 10. 11.]
 [ 3.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
 [ 4.  3.  2.  3.  4.  5.  6.  7.  8.  9.]
 [ 5.  4.  3.  2.  3.  4.  5.  6.  7.  8.]
 [ 6.  5.  4.  3.  2.  3.  4.  5.  6.  7.]
 [ 7.  6.  5.  4.  3.  2.  3.  4.  5.  6.]
 [ 8.  7.  6.  5.  4.  3.  2.  3.  4.  5.]
 [ 9.  8.  7.  6.  5.  4.  3.  2.  3.  4.]
 [10.  9.  8.  7.  6.  5.  4.  3.  2.  3.]
 [11. 10.  9.  8.  7.  6.  5.  4.  3.  2.]
 [12. 11. 10.  9.  8.  7.  6.  5.  4.  3.]
 [13. 12. 11. 10.  9.  8.  7.  6.  5.  4.]
 [14. 13. 12. 11. 10.  9.  8.  7.  6.  5.]
 [15. 14. 13. 12. 11. 10.  9.  8.  7.  6.]]


### Dynamic programming
上面印出過程中的 Dynamic programming array   
根據公式(剛好costs1給的都是2) :  
![](https://i.imgur.com/aQt1h2y.png)  
  
### Backtracking
利用公式倒推回去  
看看有沒有符合的路徑  
再以隨機的方式挑選一條走下去

## Weighted 

In [9]:
print(mineditdis_w("acknowledgement","knowledge"))

a c k n o w l e d g e m e n t 
| | | | | | | | | | | | | | | 
* * k n o w l e d * * g e * * 
d d n n n n n n n d d s n d d 
cost:  6.0

[[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
 [ 1.  1.  2.  3.  4.  4.  5.  6.  7.  8.]
 [ 2.  2.  3.  3.  4.  4.  4.  5.  6.  7.]
 [ 3.  2.  2.  3.  4.  4.  5.  6.  7.  7.]
 [ 4.  3.  2.  2.  3.  4.  5.  6.  7.  8.]
 [ 5.  4.  3.  2.  2.  3.  4.  5.  6.  7.]
 [ 6.  5.  4.  3.  2.  2.  3.  4.  5.  6.]
 [ 7.  6.  5.  4.  3.  2.  2.  3.  4.  5.]
 [ 8.  7.  6.  5.  4.  3.  2.  3.  4.  4.]
 [ 9.  8.  7.  6.  5.  4.  3.  2.  3.  4.]
 [10.  9.  8.  7.  6.  5.  4.  3.  2.  3.]
 [11. 10.  9.  8.  7.  6.  5.  4.  3.  2.]
 [12. 11. 10.  9.  8.  7.  6.  5.  4.  3.]
 [13. 12. 11. 10.  9.  8.  7.  6.  5.  4.]
 [14. 13. 12. 11. 10.  9.  8.  7.  6.  5.]
 [15. 14. 13. 12. 11. 10.  9.  8.  7.  6.]]


### Dynamic programming
在斜對角的計算中  
兩個字母的交互改以查表計算
  
### Backtracking
利用公式倒推回去  
看看有沒有符合的路徑  
再以隨機的方式挑選一條走下去
跟上面不同的是  
要確認加權數字大小  
才能確定可不可以從 sub 那條路過來